# 1.0 Подготовка датасета
## 1.1 Составление выборки
 Проанализировав предоставленные данные, я выбал из них только data 4, потому что они больше всего подходили под поставленную задачу, и выбрал от туда лишь картинки с аннотациями.

In [1]:
# Импортируем модуль os для работы с операционной системой
import os
# Получаем список файлов из папки с анотациями
l_labels = os.listdir(r'given datas\Data4\Data4\fullAnnotations')
# Сортируем его
l_labels.sort()
# Создаем пустые списки, в которые перенесем пути к файлам
images = []
labels = []
# цикл проходиться по названиям файлов лейблов и заносит пути к файлам изображений и лейблов с таким же названием
for i in l_labels:
    # добавление путей в списки
    images.append(os.path.join(r'given datas\Data4\Data4\full', i[:-4] +'.jpg'))
    labels.append(os.path.join(r'given datas\Data4\Data4\fullAnnotations', i[:-4] + '.txt'))

## 1.2 Распределение данных на train, validation и test.

In [2]:
# Из библиотеки с готовыми алгоритмами для машинного обучения импортируем модуль для разбиения выборок данных
from sklearn.model_selection import train_test_split
# Импортиурем бибилиотеку для работы с массивам данных
import numpy as np
# Разбиваем нашу обучающую выборку на train и val в соотношении 0.8 и 0.2 относительно общего обьема
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
# Разбиваем выборку val пополам на test и val в соотношении 0.1 и 0.1 относительно общего обьема
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size=0.5, random_state=42)
# Выводим информацию о получившихся выборках train, val, test:
print(f'Количество элементов в train выборке: {np.array(train_images).size}')
print(f'Количество элементов в val выборке: {np.array(val_images).size}')
print(f'Количество элементов в test выборке: {np.array(test_images).size}')

Количество элементов в train выборке: 4720
Количество элементов в val выборке: 590
Количество элементов в test выборке: 590


## 1.3 Создание структуры датасета

In [60]:
# Создаем переменную, в котрой будет храниться название директорий датасета
dataset= 'dataset'
# Созаем кортеж, в котором будем хранить названия всех путей датасета
dirs = ('images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test')
# Создаем цикл, проверяющий наличие данных директорий, при их отсуствие он их создает
for dir in dirs:
    # Создаем путь до директории датасета
    path_dir = os.path.join(dataset, dir)
    # Если директория еще не создана, то создаем ее
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)

## 1.4 Копирование файлов из data 4 в dataset

In [61]:
# Импортируем функцию для копирования файлов
from shutil import copy
# Создаем функцию для копирования файлов из одной директории в другую
def move_files(files, destination):
    # Цикл проходиться по всем файлам из переменной
    for file in files:
        # Пытаемся копировать, если происходит ошибка, то выводится в консоль сообщение об ошибке
        try:
           copy(file, destination)
        except:
            print(f'Ошибка копирования: {file}')
# Создаем двухмерный массив со всеми файлами внтури
all_files = [[train_images, val_images, test_images], [train_labels, val_labels, test_labels]]
# Создаем кортеж с переменнами нашей выборки
variabels  = ('images', 'labels')
# Создаем кортеж с задачами выборки
tasks = ('train', 'val', 'test')
# Создаем цикл, который копирует файлы используя написанную выше функцию
for i in range(len(variabels)):
    path = os.path.join(dataset, variabels[i])
    for j in range(len(tasks)):
        move_files(all_files[i][j], os.path.join(path, tasks[j]))

# 2.0 Создание файла data.yaml
### 2.1 Создаем в директории файл data.yaml
### 2.2 Передаем в этот файл пути до наших выборок: train, val, test
### 2.3 Передаем количество классов и их имена в файл

При необхоимости нужно заменить путь до нашего датасета в settings.yaml по данному пути: "C:\Users\Server\AppData\Roaming\Ultralytics"

# 3.0 Обучение модели
### 3.1 Загрузка предобученной модели YOLO

In [10]:
# Импортиумем из библиотеки ultralytics модуль YOLO
from ultralytics import YOLO
# Загружаем предобученную модель yolov8n.pt (версия - 8, размер - nano)
model = YOLO('yolov8n.pt')


### 3.2 Тренировка модели

In [11]:
# Задаем параметр обучения модели и обучаем ее
results = model.train(data='data.yaml', epochs=4, optimizer = 'AdamW', name = 'yolo')


Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.1+cpu CPU (Intel Core(TM) i7-9700T 2.00GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=4, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning C:\Users\Ws2024\Desktop\Мой проект\Модуль А\dataset\labels\train.cache... 4720 images, 60 backgrounds, 0 corrupt: 100%|██████████| 4720/4720 [00:00<?, ?it/s]
val: Scanning C:\Users\Ws2024\Desktop\Мой проект\Модуль А\dataset\labels\val.cache... 590 images, 12 backgrounds, 0 corrupt: 100%|██████████| 590/590 [00:00<?, ?it/s]

Plotting labels to runs\detect\yolo\labels.jpg... 


optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\yolo
Starting training for 4 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/4         0G       1.16      1.002      1.137         30        640: 100%|██████████| 295/295 [29:21<00:00,  5.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:04<00:00,  3.41s/it]

...

        4/4         0G     0.9463     0.5812      1.053         38        640: 100%|██████████| 295/295 [31:41<00:00,  6.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:08<00:00,  3.61s/it]

                   all        590        660      0.971      0.935      0.966      0.749



4 epochs completed in 2.167 hours.
Optimizer stripped from runs\detect\yolo\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolo\weights\best.pt, 6.2MB

Validating runs\detect\yolo\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.8 torch-2.2.1+cpu CPU (Intel Core(TM) i7-9700T 2.00GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:57<00:00,  3.02s/it]


                   all        590        660       0.97      0.935      0.966      0.749
Speed: 2.6ms preprocess, 84.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\yolo
